In [0]:
## ※ contrib.은 접어라(버전은 니가 업데이트해라)
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
## 이전 강의 예제 복습 
## 강사님 예제풀이 - 기존 설명위주의 코드에서 좀 더 유려한 것으로 
## 지금까지 배운 내용을 기초로 tensorflow의 mnist example을 CNN으로 구현해보자 
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np 

# Data Loading 
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## 전처리 단계(결측치, 이상치, 정규화, feature engineering)
## mnist 예제에서는 이미 전처리가 끝난 상태여서 따로 할 건 없어요! 

# Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
#※ tensorflow 버전에 따라서 dropout을 사용하는 속성이 변경 
#  최신버전(버리는쪽) -> rate =0.8 : drop 시키는 노드의 비율 
#  이전버전(살리는쪽) -> keep_prob = 0.8 : 살리는 노드의 비율  
keep_rate = tf.placeholder(dtype=tf.float32)

# 2.2 Convolution Layer 
# 입력데이터는 4차원 
x_img = tf.reshape(X,[-1,28,28,1])
L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                      kernel_size = [3,3], 
                      padding = "SAME", #기존 적었던 내용들을 함수의 인자들로
                      strides = 1, 
                      activation =tf.nn.relu)   
L1 = tf.layers.max_pooling2d(inputs = L1, 
                             pool_size=[2,2],
                             padding = "SAME", 
                             strides=2)
L1 = tf.layers.dropout(inputs = L1, rate= 1- keep_rate)
'''이하 주석와 동일
W1 = tf.Variable(tf.random_normal([3,3,1,32])) #채널수(1)는 항상 일치시켜야 conv이 발생한다 

L1 = tf.nn.conv2d(x_img,W1,strides=[1,1,1,1], 
                  padding = "SAME") #합성곱 연산단계
L1 = tf.nn.relu(L1) #relu단계 
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding="SAME") #pooling 단계
'''


L2 = tf.layers.conv2d(inputs=L1, filters=64,
                      kernel_size = [3,3], padding = "SAME", #기존 적었던 내용들을 함수의 인자들로
                      strides = 1, activation =tf.nn.relu)   
L2= tf.layers.max_pooling2d(inputs = L2, pool_size=[2,2],
                             padding = "SAME", strides=2)
L2= tf.layers.dropout(inputs = L2, rate= 1 - keep_rate)
'''이하 주석과 동일 
W2 = tf.Variable(tf.random_normal([3,3,32,64])) #32는 받는것이므로 고정, 64는 내가 정한 임의값 

L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1], 
                  padding = "SAME") #합성곱 연산단계
L2 = tf.nn.relu(L2) #relu단계 
L1 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding="SAME") #pooling 딘계
'''

# 2.3 FC layer (=dense layer)
# dense 함수 설정하면 세줄짜리 코드를 한줄분량으로 설정 가능 
L2 = tf.reshape(L2,[-1,7*7*64])
dense1 = tf.layers.dense(inputs = L2, units=256, 
                         activation = tf.nn.relu)
dense1 = tf.layers.dropout(inputs=dense1, rate= 1 - keep_rate)

dense2= tf.layers.dense(inputs = dense1, units=128, 
                         activation = tf.nn.relu)
dense2= tf.layers.dropout(inputs=dense2, rate= 1 - keep_rate)

dense3 = tf.layers.dense(inputs = dense2, units=512, 
                         activation = tf.nn.relu)
dense3 = tf.layers.dropout(inputs=dense3, rate= 1 - keep_rate)
'''이하 주석과 동일 
W3 = tf.get_variable("weight3", shape=[7*7*64, 256],
                    initializer = tf.contrib.layers.xavier_initializer())#Xavier쓸거니깐 get_variable선언
b3 = tf.Variable(tf.random_normal([256]), name="bias3")
_layer1 = tf.nn.relu(tf.matmul(L2,W3) + b3)
layer1 = tf.nn.dropout(_layer1, keep_prob = dropout_rate)

W4 = tf.get_variable("weight4", shape=[256,512],
                    initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]), name="bias4")
_layer2 = tf.nn.relu(tf.matmul(layer1,W4) + b3)
layer2 = tf.nn.dropout(_layer2, keep_prob = dropout_rate)
'''

#Hypothesis 
H = tf.layers.dense(inputs = dense3, units=10)
'''이하 주석과 동일                     
W5 = tf.get_variable("weight5", shape=[512,10],
                    initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]), name="bias5")

logit = tf.matmul(layer2, W5) + b5
H = tf.nn.softmax(logit)
'''
                    
#Cost
cost = tf.losses.softmax_cross_entropy(Y,H)
'''이하 주석과 동일 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits= logit
                                                                 labels= Y))
'''

#train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

#session 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습 
num_of_epoch = 30 
batch_size = 100 

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples / batch_size)
    cost_val = 0 # 코스트 초기화(국룰)
    
    for i in range(num_of_iter):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x, 
                                                        Y:batch_y,
                                                        keep_rate:0.7 })
    if step % 3 == 0 :
        print("Cost는 : {}".format(cost_val))

#정확도
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print("정확도는 : {}".format(sess.run(accuracy, 
                                   feed_dict={X:mnist.test.images, 
                                              Y:mnist.test.labels,
                                              keep_rate:1.0})))
#prediction


In [0]:
###Ensemble(앙상블)
## 결국 우리의 MNIST예제는 multinomial 예제
## 이미지 1개에 대한 예측값이 H의 도출값은 [0.5, 0.3, 0.2, 0.001, 0.99, 0.44 ..] 
## 원소를 다 더하면 1이 나오는 예측값. 

## ensenble은 모델을 여러개 만든다. (10개의 모델)
## 이미지 1개에 대한 각 모델의 예측값 
## H1 -> [0.5, 0.3, 0.2, 0.001, 0.99, 0.44 ...]
## H3 -> [0.4, 0.2, 0.3, 0.011, 0.90, 0.64 ...]
## H3 -> [0.8, 0.3, 0.4, 0.005, 0.78, 0.84 ...]
## Hn -> [...]

## SUM -> [1.7, 0.8, 0.9, 0.017, ...]
## 동일한 학습 알고리즘을 사용해 여러(두 가지 이상)모델을 학습하는 형태 
## 한 사람의 전문가한테 의견을 구하는 게 아니라 
## 여러사람의 전문가에게 의견을 구해서 가장 그럴듯한 대답을 찾아내는 방식 

## 앞선 mnist multinomial 을 재활용해보자! 

In [0]:
## 다완's
##tensorflow의 mnist example을 CNN으로 구현해보자 
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np 

# Data Loading 
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## 전처리 단계(결측치, 이상치, 정규화, feature engineering)
## mnist 예제에서는 이미 전처리가 끝난 상태여서 따로 할 건 없어요! 

# Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
#※ tensorflow 버전에 따라서 dropout을 사용하는 속성이 변경 
#  최신버전(버리는쪽) -> rate =0.8 : drop 시키는 노드의 비율 
#  이전버전(살리는쪽) -> keep_prob = 0.8 : 살리는 노드의 비율  
keep_rate = tf.placeholder(dtype=tf.float32)

# 2.2 Convolution Layer
# 입력데이터는 4차원!
x_img = tf.reshape(X,[-1,28,28,1])  # 입력데이터 
L1 = tf.layers.conv2d(inputs=x_img, filters = 32,
                     kernel_size = [3,3],
                     padding = 'SAME',
                     strides = 1,
                     activation = tf.nn.relu)
# W1 = tf.Variable(tf.random_normal([3,3,1,32]))                   # 필터
# L1 = tf.nn.conv2d(x_img,W1, strides=[1,1,1,1],                   # 합성곱 작업!
#                   padding="SAME")
# L1 = tf.nn.relu(L1)
# 이만큼의 작업을 위처럼 바꾸면 한번에 처리!

L1 = tf.layers.max_pooling2d(inputs=L1,
                            pool_size = [2,2],
                            padding = 'SAME',
                            strides = 2)

L1 = tf.layers.dropout(inputs=L1, rate = 1-keep_rate)

L2 = tf.layers.conv2d(inputs=L1, filters = 64,
                     kernel_size = [3,3],
                     padding = 'SAME',
                     strides = 1,
                     activation = tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs=L2,
                            pool_size = [2,2],
                            padding = 'SAME',
                            strides = 2)

L2 = tf.layers.dropout(inputs=L2, rate = 1-keep_rate)



# # 2.3 FC Layer (Dense Layer)
L2 = tf.reshape(L2,[-1,7*7*64])
dense1 = tf.layers.dense(inputs = L2, units = 256,
                        activation = tf.nn.relu)
# dense함수 이용하면 아래 3줄짜리 코드를 위처럼 한 줄로 설정 가능!!

# W3 = tf.get_variable('weight3',shape=[7*7*64,256],
#                     initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.random_normal([256]),name = 'bias3')
# _layer1 = tf.nn.relu(tf.matmul(L2,W3)+b3)
dense1 = tf.layers.dropout(inputs = dense1, rate = 1-keep_rate)
# layer1 = tf.nn.dropout(_layer1, keep_prob=keep_rate)

dense2 = tf.layers.dense(inputs = dense1, units = 128,
                        activation = tf.nn.relu)
dense2 = tf.layers.dropout(inputs = dense2, rate = 1-keep_rate)

dense3 = tf.layers.dense(inputs = dense2, units = 512,
                        activation = tf.nn.relu)
dense3 = tf.layers.dropout(inputs = dense3, rate = 1-keep_rate)
# W4 = tf.get_variable('weight4',shape=[256,512],
#                     initializer=tf.contrib.layers.xavier_initializer())
# b4 = tf.Variable(tf.random_normal([512]),name='bias4')

# Hypothesis
H = tf.layers.dense(inputs= dense3, units = 10)
# Cost
cost = tf.losses.softmax_cross_entropy(Y,H)  # 순서 바뀐거 중요!!

train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

# 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 ( 데이터 사이즈가 커졌기 때문에 batch 필요! )

num_of_epoch = 30
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples / batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        _,cost_val = sess.run([train, cost],feed_dict = {X:batch_x,
                                                         Y:batch_y,
                                                        keep_rate:0.7})
    if step % 3 == 0:
        print('cost : {}'.format(cost_val))
        
# predition, accuracy측정

predict = tf.argmax(H,1)
correct = tf.equal(predict,tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

print('정확도 ;{}'.format(sess.run(accuracy,feed_dict={X: mnist.test.images,
                                                   Y:mnist.test.labels,
                                                   keep_rate:1.0})))


#ensamble(10번)
for i in range(10):
    tf.reset_default_graph() #런
    num_of_epoch = 30
    batch_size = 100
    
    for step in range(num_of_epoch):
            num_of_iter = int(mnist.train.num_examples / batch_size)
            cost_val = 0
            for j in range(num_of_iter):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                _,H_val,cost_val = sess.run([train,H,cost], feed_dict={X : batch_x,
                                                       Y : batch_y, 
                                                       keep_rate : 0.7})
    print(cost_val)    
    H_val = sess.run(H, feed_dict={X: mnist.test.images})
    if i == 0:
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)

predict = np.argmax(H_result,1)
Y = mnist.test.labels
correct = np.equal(predict, np.argmax(Y,1))
accuracy = np.mean(correct)
print("앙상블 정확도:", accuracy)